In [ ]:
'''

testing

estimated long axis images (png file)

'''

import pickle
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import copy
import imageio
import skimage
import random
import tensorflow as tf

from glob import glob
from PIL import Image
from tensorflow import keras
from skimage import io
from shutil import copyfile, copytree

from dl_func_la import gen_cnn_model, grayscale_to_rgb, resize_to_224

gpu_no = '/gpu:0'

base_dir = r'D:\python\yoonckim\longaxis'
excel_file = 'label_motion_10_bmcR1.xlsx'  # 

# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\CustomCBR\noAugment_test' 
# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\CustomCBR\flipLR_test' 

model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\EfficientNetB0\noAugment_test' 
# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\EfficientNetB0\flipLR_test'

# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\MobileNet\noAugment_test' 
# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\MobileNet\flipLR_test' 

# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\NASNetMobile\noAugment_test' 
# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\NASNetMobile\flipLR_test' 

# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\ResNet50\noAugment_test' 
# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\ResNet50\flipLR_test' 

# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\VGG16\noAugment_test' 
# model_dir = r'D:\python\yoonckim\longaxis\data\CAT_KAG_model1\VGG16\flipLR_test' 


png_test_dir = r'data\CAT_KAG\test'    
dir_test = r'data\CAT_KAG\test_for_eval'  

fname1 = os.path.join(base_dir, excel_file)
df1 = pd.read_excel(fname1, sheet_name='Sheet1', index_col=None, engine='openpyxl')
print(df1['ID'].size)
df2 = df1[df1['image_interpretable'] == 'interpretable'] 
print(df2['ID'].size)
a = list(df2.columns)
col_list = a
print(col_list)

id_list = list(df2['ID'])
png_fname_list = list(df2['png_fname'])
# isTrain_list = list(df2['isTrain'])
motion_flag_list = list(df2['motion_flag'])
# print(motion_flag_list)


In [ ]:

if False:
    
    list_dirs = glob(png_test_dir + '\\*')
    nsubj = len(list_dirs)
    print(f'nsubj = {nsubj}')
    dir_, id_ = os.path.split(list_dirs[0])

    for kk, dir_name in enumerate(list_dirs):
        dir_, id_folder = os.path.split(dir_name)
        subj_png_dir2 =  os.path.join(dir_test, id_folder)
        dir_name2 = os.path.join(dir_name, 'estLA')
        copytree(dir_name2, subj_png_dir2)  
    

In [ ]:


def find_index_string(png_fname_list1, substring):
    
    for ind, fname_ in enumerate(png_fname_list1):
        if substring in fname_:
            index = ind
            break
        else:
            index = -1000 
    
    return index


In [ ]:


showImage = True

test_img = []
test_motion = []  

dir_id = glob(dir_test + '\\*')

print(f'number of test subjects = {len(dir_id)}')
print(dir_id[0])
print(model_dir)
fname_model_list = glob(model_dir + '\\CNN_*.h5')

model_list = []
for _, fname_model in enumerate(fname_model_list):
    model = keras.models.load_model(fname_model, compile=False)
    model_list.append(model)

pred_per_img_all = []
label_per_img_all = []
    
for kk, id_ in enumerate(dir_id): 
    print('kk = %d, id = %s' % (kk, id_))
  
    _, id_only = os.path.split(id_)
    
    list_png = glob(id_ + '\\*.png')
    input_img = []
    label_img = []
    fname_img = []
    
    for ll, fname_png in enumerate(list_png): 
        img = imageio.imread(fname_png)
        img2 = skimage.transform.resize(img, (96, 256), anti_aliasing=True, preserve_range=False)
        _, fname_ = os.path.split(fname_png)
        
        img2 = img2[:, (128-96):(128+96)]  
        img2 = img2/np.max(img2)
        
        ind = find_index_string(png_fname_list, fname_)
        if ind == -1000:
            print(f'{fname_} is outlier. continue')
            continue
        
        if 'presen' in motion_flag_list[ind]: 
            motion_eLA = 1
        elif 'absen' in motion_flag_list[ind]: 
            motion_eLA = 0
        else: 
            quit()
    
        input_img.append(img2)
        label_img.append(motion_eLA)
        fname_img.append(fname_)
    
    X_data = np.stack(input_img, axis=0)
    y_data = np.stack(label_img, axis=0)
    
    X_data_ = grayscale_to_rgb(X_data)

    sess2 = tf.compat.v1.Session()
    with sess2.as_default():
        X_data = X_data_.numpy()
    print(X_data.shape)

    if not 'CustomCBR' in model_dir:
        print('Transfer learning - pretrained model')
        X_data = resize_to_224(X_data)
    else:
        print('Custom CBR')
    
    nsamp = X_data.shape[0]
    print(nsamp)
    prob_score_list = []
    for _, model in enumerate(model_list):
        prob_score = model.predict(X_data, batch_size=len(input_img), verbose=0)
        prob_score_list.append(prob_score)
    
    arr_prob_score = np.dstack(prob_score_list)
    arr_prob_score_mean = np.mean(arr_prob_score, axis=2)
    arr_prob_score_mean_mean = np.mean(arr_prob_score_mean, axis=0)
    print(f'ID = {id_only}')
     
    pred_per_img_all.append(arr_prob_score_mean)
    label_per_img_all.append(y_data)
    nLA = X_data.shape[0]
    
    if showImage==True:
        for jj in range(nLA):
            plt.figure()
            plt.imshow(X_data[jj,:,:,1], cmap='gray')
            plt.title(f'fname={fname_img[jj]}, label={y_data[jj]}\n pred={arr_prob_score_mean[jj, :]}')
        plt.show()
    
pred_per_img_all = np.vstack(pred_per_img_all)
label_per_img_all = np.hstack(label_per_img_all)

print(pred_per_img_all.shape)
print(label_per_img_all.shape)



In [ ]:

from sklearn import metrics

fpr1, tpr1, thresh1 = metrics.roc_curve(label_per_img_all, pred_per_img_all[:, 1], pos_label=1)
roc_auc_1 = metrics.auc(fpr1, tpr1)
ind1 = np.argmax(tpr1-fpr1)
print(f'Probability cutoff = {thresh1[ind1]}')

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
lw = 3
plt.plot(fpr1, tpr1, color='darkorange', alpha=1.0, lw=lw, label='IBH Motion Prediction: AUC = %0.2f' % roc_auc_1)

plt.xlim([-0.05, 1.0])
plt.ylim([-0.05, 1.05])
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.title('ROC')
plt.legend(loc="lower right")
plt.grid(True)

ax.set_aspect('equal')

plt.show()
